In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

nltk.download('punkt')
nltk.download('stopwords')


df = pd.read_excel('input.xlsx')
# print(df.head)
for ind in df.index:
    url = df['URL'][ind]
    url_id = df['URL_ID'][ind]
    response = requests.get(url)
# print(response)
    soup = BeautifulSoup(response.content, 'html.parser')
# print(soup)
    Title = soup.title.get_text()
# print(Title)
    f = open('%s.txt' % url_id, 'w', encoding="utf-8")
    f.write(Title + "\n")
    for data in soup.find_all("p"):
        Text = data.get_text()
        f.writelines(Text + "\n")
    f.close()
    
text = open('42146.8.txt',encoding="utf-8").read()
lower_case =text.lower().translate(str.maketrans('','',string.punctuation))
tokenized_words = word_tokenize(lower_case,'english')
# print(tokenized_words)
final_words = []

for words in tokenized_words:
    if words not in stopwords.words('english'):
        final_words.append(words)
# gettting text and title of input file in final_words list
# print(final_words)     

# files to match positive anda negative feed
positive_words_file = open('positive-words.txt',encoding="utf-8").read()
negative_words_file = open('negative-words.txt',encoding="utf-8").read()

# defining variable to store values of parameters needed in output file
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []
avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

# calculating each varibles
positive_words.append([word for word in final_words if word.lower() in positive_words_file])
# 
Negative_words.append([word for word in final_words if word.lower() in negative_words_file])
# 
positive_score.append(len(positive_words))
# 
negative_score.append(len(Negative_words))
# 
polarity_score.append((positive_score - negative_score) / ((positive_score + negative_score) + 0.000001))
# 
subjectivity_score.append((positive_score + negative_score) / ((len(final_words)) + 0.000001))
# 
complex_words = []
for word in final_words:
    vowels = 'aeiou'
    syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
    if syllable_count_word > 2:
        complex_words.append(word)
complex_words_count = len(complex_words)
# 
syllable_count = 0
syllable_words =[]
for word in final_words:
    if word.endswith('es'):
        word = word[:-2]
    elif word.endswith('ed'):
        word = word[:-2]
    vowels = 'aeiou'
    syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
    if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word
        
sentences = final_words.split('.')
num_words = len(final_words)
num_sentences = len(sentences)
avg_sentence_len = num_words / num_sentences
avg_syllable_word_count = syllable_count / len(syllable_words)
Percent_Complex_words  =  len(complex_words) / num_words
Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

# 
word_count = []
words = final_words.split('')
word_count = len(words)
# 
personal_pronouns = ["I", "we", "my", "ours", "us"]
count = 0
for pronoun in personal_pronouns:
    count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
# print(count)
# 
length = sum(len(word) for word in final_words)
average_word_length = length / len(final_words)

# dataframe 
outp_df = pd.read_excel('Output Data Structure.xlsx')

variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            count,
            average_word_length]

# substituting values in file
for index, value in enumerate(variables):
  outp_df.iloc[:,index+2] = value

#saving file
outp_df.to_csv('Result.csv')
